# Chapter 12: Generative UI

Generative UI represents an approach to rendering dynamic and interactive elements within chat windows or applications. This technique leverages the capabilities of LLMs to generate and manage various user interface components - such as images, tables, maps, and cards-on-the-fly. Generative UI offers several methods for achieving dynamic content generation:
- **Code generation and rendering:** In this approach, the LLM writes code to create visual elements. For instance, it could generate code to create a chart or image based on data analysis. The resulting code is executed to render the content, and the generated visual may be sent as binary data to the UI for display.
- **Tool-based rendering:** Here, rendering is driven by the tools invoked by the LLM. After a specific tool generates the desired output, the resulting data is sent to the frontend for rendering. Typically, you focus on rendering the output for the most recent tool called, rather than displaying multiple widgets. This approach ensures that the UI remains dynamic and responsive, showing only the most relevant and updated information, thus avoiding clutter and maintaining a clean and interactive user experience.
- **Dynamic UI updates:** This technique involves asking the LLM to produce an output object that defines how the UI should be updated. For example, the LLM could generate an object that specifies new filters for charts or other interactive elements. This output is sent back to the frontend to dynamically update the user interface based on the latest data or user inputs.

Generative UI enhances user experiences by using LLMs to manage dynamic content. In this discussion, we will focus on the second approach: tool-based rendering.

![generative-ui](./assets/images/generative-ui.gif)

In [1]:
from assets.tools.earthquake import count_earthquakes, query_earthquakes, USGeopoliticalSurveyEarthquakeAPI
from language_models.agent import (
    Agent,
    OutputType,
    PromptingStrategy,
    Step,
)
from language_models.tools import Tool, current_date
from language_models.models.llm import OpenAILanguageModel
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [2]:
proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [3]:
query_earthquakes_tool = Tool(
    function=query_earthquakes,
    name="Query Earthquakes",
    description="Use this tool to search recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

count_earthquakes_tool = Tool(
    function=count_earthquakes,
    name="Count Earthquakes",
    description="Use this tool to count and aggregate recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

system_prompt = "You are an United States Geological Survey expert who can answer questions regarding earthquakes."

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-4",
    max_tokens=500,
    temperature=0.2,
)

agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{question}",
    prompt_variables=["question"],
    output_type=OutputType.STRING,
    tools=[current_date, count_earthquakes_tool, query_earthquakes_tool],
    prompting_strategy=PromptingStrategy.CHAIN_OF_THOUGHT,
    verbose=True,
)

In [4]:
output = agent.invoke({"question": "How many earthquakes occurred today?"})

Thought: To answer this question, I need to use the "Count Earthquakes" tool. I need to set the start_time to the start of today and the end_time to the current time.
Tool: Current Date
Tool Input: {}
Tool Output: 2024-08-21 09:22:27.847782
Thought: Now that I have the current date and time, I can use the "Count Earthquakes" tool to find out how many earthquakes occurred today. I will set the start_time to the start of today (2024-08-21 00:00:00) and the end_time to the current time (2024-08-21 09:22:27.847782).
Tool: Count Earthquakes
Tool Input: {'start_time': '2024-08-21 00:00:00', 'end_time': '2024-08-21 09:22:27.847782'}
Tool Output: {'count': 89, 'maxAllowed': 20000}
Thought: The tool has provided the number of earthquakes that occurred today. I can now provide this information to the user.
Final Answer: There have been 89 earthquakes today.


In [5]:
print(output.final_answer)

There have been 89 earthquakes today.


In [6]:
output = agent.invoke({"question": "Show me 3."})

Thought: To show the details of 3 earthquakes that occurred today, I need to use the "Query Earthquakes" tool. I will set the start_time to the start of today, the end_time to the current time, and the limit to 3.
Tool: Query Earthquakes
Tool Input: {'start_time': '2024-08-21 00:00:00', 'end_time': '2024-08-21 09:22:27.847782', 'limit': 3}
Tool Output: {'type': 'FeatureCollection', 'metadata': {'generated': 1724224970000, 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-08-21+00%3A00%3A00&endtime=2024-08-21+09%3A22%3A27.847782&limit=3&mindepth=-100&maxdepth=1000&eventtype=earthquake', 'title': 'USGS Earthquakes', 'status': 200, 'api': '1.14.1', 'limit': 3, 'offset': 1, 'count': 3}, 'features': [{'type': 'Feature', 'properties': {'mag': 1.9, 'place': '6 km S of Volcano, Hawaii', 'time': 1724224190890, 'updated': 1724224348610, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/hv74416161', 'detail': 'https://earthquake.usgs.gov/fd

In [7]:
print(output.final_answer)

Here are the details of 3 earthquakes that occurred today:

1. A magnitude 1.9 earthquake occurred 6 km South of Volcano, Hawaii. More details can be found [here](https://earthquake.usgs.gov/earthquakes/eventpage/hv74416161).

2. A magnitude 1.91 earthquake occurred 9 km South-Southwest of Volcano, Hawaii. More details can be found [here](https://earthquake.usgs.gov/earthquakes/eventpage/hv74416156).

3. A magnitude 2.48 earthquake occurred 3 km West of Pāhala, Hawaii. More details can be found [here](https://earthquake.usgs.gov/earthquakes/eventpage/hv74416151).
